In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 2.6 MB/s eta 0:00:00


In [2]:
## Load modules and models

import torch  # load pytorch
import torchvision
import xlsxwriter

In [3]:
## Set up model for QuestionAnswering and taking tokens

from transformers import BertForQuestionAnswering ## load the model for QA
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

from transformers import BertTokenizer ## load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
## define the function for answering

def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    ## define special punctuations
    pun_left  = ['.', ')'] # punctuations that need to remove the left space
    pun_right = ['$', '('] # punctuations that need to remove the right space
    pun_both  = ['-', '/'] # punctuations that need to remove the both side spaces
    pun_special_1 = [','] # need to remove the left space, but if in a $ express, need to remove both side spaces.
    pun_special_2 = ['’'] # if follows by s, need to remove both side, otherwise, only need to remove left space

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        elif tokens[i] in pun_left:
            answer += tokens[i] ## no space in the left side

        elif tokens[i] in pun_both:
            answer += tokens[i] ## no space in the left side

        elif tokens[i] in pun_special_1:
            answer += tokens[i] ## no space in the left side

        elif tokens[i] in pun_special_2:
            answer += tokens[i] ## no space in the left side

        # Otherwise, add a space then the token.
        else:
            if i == 0:
                answer += tokens[i]
            else:
                if tokens[i-1] in pun_both:
                    answer +=  tokens[i]  ## no space in the left side of token[i]

                elif tokens[i-1] in pun_right:
                    answer +=  tokens[i]  ## no space in the left side of token[i]

                elif tokens[i-1] in pun_special_1:
                    if '$' in tokens[max(0, i-5):i]:  ## ',' in $ expression
                        answer +=  tokens[i]
                    else:
                        answer += ' ' + tokens[i]

                elif tokens[i-1] in pun_special_2:
                    if tokens[i] == 's':  ## ’s expression
                        answer +=  tokens[i]
                    else:
                        answer += ' ' + tokens[i]

                else:
                    answer += ' ' + tokens[i]


    # Extract numerical value (handles cases like "$100", "100 dollars", etc.)
    match = re.search(r'(\$?\d+[\d,]*\.?\d*)', answer)
    if match:
        return match.group(1)  # Return only the number (with $ if present)
    else:
        return answer  # Default if no number is found

    #return(answer)


In [5]:
## define the function for computing probability

def answer_question_probability(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    ## convert tensor type to list, and then maximum it
    probability = max(max(start_scores.tolist())) + max(max(end_scores.tolist())) # they are scores, so add them together.

    return(probability)

In [6]:
## define the function to compute the index of high probability for long file

def answer_question_longfile_probability(question, answer_text, max_num_sentences, overlap_num_sentences):

    ## First, split the answer text into sentences.
    content = answer_text.splitlines()

    ## get number of sentences
    len_sentence = len(content)

    ## how many pieces we will split, (get the ceiling of the division) + 1
    piece_num = - ( - (len_sentence -  max_num_sentences) // (max_num_sentences - overlap_num_sentences) ) + 1

    ## use prob to save all probilities
    prob = [0] * piece_num

    ##-------------------------------------------------##
    ## compute the probability of the first piece
    ##-------------------------------------------------##

    ## get the first piece
    content_temp = " "

    for ii in range(max_num_sentences):
        content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

    ## compute the probability for this piece (prob(start) * prob(end))
    prob[0] = answer_question_probability(question, content_temp)  # probability for the first piece

    ##------------------------------------------------------------##
    ## if only split into 2 pieces, go the last piece directly
    ##------------------------------------------------------------##

    if piece_num == 2:
        content_temp = " "

        for ii in range(max_num_sentences - overlap_num_sentences, len_sentence):
            content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

        ## compute the probability for this piece (prob(start) * prob(end))
        prob[1] = answer_question_probability(question, content_temp) # compute the probability for the last piece

    else: ## if split into more than 2 pieces

        for jj in range(1, piece_num - 1): ## compute pieces between the first one and the last one.

            content_temp = " "

            for ii in range((max_num_sentences - overlap_num_sentences)*jj, (max_num_sentences - overlap_num_sentences)*jj + max_num_sentences):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## compute the probability for this piece (prob(start) * prob(end))
            prob[jj] = answer_question_probability(question, content_temp) # compute the probability for the last piece



        ## compute the last piece
        content_temp = " "

        for ii in range((max_num_sentences - overlap_num_sentences)*(piece_num - 1), len_sentence):
            content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

        ## compute the probability for this piece (prob(start) * prob(end))
        prob[piece_num - 1] = answer_question_probability(question, content_temp) # compute the probability for the last piece

    max_prob   = max(prob)  ## get the highese probability
    max_index  = prob.index(max_prob) ## find the index

    return(max_index)

In [7]:
## define the function for splitting the files

def answer_question_longfile(question, answer_text, max_num_sentences, overlap_num_sentences):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    ## if len(input_ids) < 512, this function goes to answer_question directly, otherwise we split the file into multiple sub-files.
    if len(input_ids) <= 512:

        answer = answer_question(question, answer_text)

       # print('Question: "' + question + '"')
       # print('Answer: "' + answer + '"')


    else:
        ## get the index of piece with the highest probability.
        max_index = answer_question_longfile_probability(question, answer_text, max_num_sentences, overlap_num_sentences)

        ##---------------------------------##
        ## get piece numbers               ##
        ##---------------------------------##
        ## First, split the answer text into sentences.
        content = answer_text.splitlines()

        ## get number of sentences
        len_sentence = len(content)

        ## how many pieces we will split, (get the ceiling of the division) + 1
        piece_num = - ( - (len_sentence -  max_num_sentences) // (max_num_sentences - overlap_num_sentences) ) + 1

        ## if it is the first piece
        if max_index == 0:
            ## get the piece
            content_temp = " "

            for ii in range(max_num_sentences):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## get the answer

            answer = answer_question(question, content_temp)

            # print('Question: "' + question + '"')
            # print('Answer: "' + answer + '"')


        ## if it is the last piece
        elif max_index == piece_num - 1:

            content_temp = " "

            for ii in range((max_num_sentences - overlap_num_sentences)*(piece_num - 1), len_sentence):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## get the answer
            answer = answer_question(question, content_temp)

            # print('Question: "' + question + '"')
            # print('Answer: "' + answer + '"')

        else: # if in a middle piece
            content_temp = " "

            for ii in range((max_num_sentences - overlap_num_sentences)*max_index, (max_num_sentences - overlap_num_sentences)*max_index + max_num_sentences):
                content_temp = content_temp + content[ii].strip() + " "  ## use " " to seperate these sentences

            ## get the answer
            answer = answer_question(question, content_temp)

            # print('Question: "' + question + '"')
            # print('Answer: "' + answer + '"')

    return(answer)



In [8]:
## split a file by paragraph

## 1, define a function to count tokens
def word_count(string):
    tokens = string.split()  # split it by tokens
    n_tokens = len(tokens)   # count how many of these tokens
    return n_tokens

## 2, define a function to split a file by paragraph, and then remove piece less than 3 tokens (blank space or title)

def split_paragraph(string):
    string_split = string.split("\n\n") # split the file by /n/n

    # get how many pieces
    len_para = len(string_split)

    paragraphs = ['']  # an empty list to contain the results

    for ii in range(len_para):
        if word_count(string_split[ii]) >= 4: # add split piece into 'paragraphs' if there are more 4 tokens
            paragraphs.append(string_split[ii])

    ## remove the first empty element in paragraphs
    paragraphs = paragraphs[1:len(paragraphs)]

    return(paragraphs)


In [9]:
# these functions are heavily influenced by the HF squad_metrics.py script
import string
import re

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers

In [10]:
## this is a function to compute EM and F1 together
import statistics

def compute_EM_F1(Prediction, Truth):
    # get how many questions
    len_para = len(Prediction)

    EM = ['']  # an empty list to contain the EM results
    F1 = ['']  # an emputy list to contain F1 results

    for ii in range(len_para):
        em_score = compute_exact_match(Prediction[ii], Truth[ii])  # compute em score one by one
        f1_score = compute_f1(Prediction[ii], Truth[ii])           # compute f1 score one by one

        EM.append(em_score) # add em score to EM
        F1.append(f1_score) # add f1 score to F1

    ## remove the first empty element in EM and F1
    EM = EM[1:len(EM)]
    F1 = F1[1:len(F1)]

    ## print the results out
    print(f"EM score: {statistics.mean(EM)}\n")
    print(f"F1 score: {statistics.mean(F1)}\n")

    return(EM, F1)

In [21]:
import pandas as pd
import os
import chardet
import zipfile

# Set up parameters and questions
Questions_item01 = [
    'What is the initial fee?',
    'What is the initial fee in dollars?',
    'What is the initial franchise fee in dollars?',
    'What is the standard nonrefundable initial franchise fee in dollars?'
]

max_num_sentences = 20
overlap_num_sentences = 5

# Create an empty list to store the answers for each file
all_answers = []
count =0

# Extract the zip file
extract_dir = "/content/"
fname = "Item5.zip"

zip_path = extract_dir + fname
with zipfile.ZipFile(zip_path, "r") as f:
    f.extractall(extract_dir+'/item5/')

# Loop through all the files
for filename in os.listdir('/content//item5/'):
    if filename.endswith(".txt"):
        file_path = os.path.join('/content/item5/', filename)

        # Open the file and read the contents
        with open(file_path, "rb") as file:
            answer_text = file.read()
            encoding = chardet.detect(answer_text)['encoding']

        with open(file_path, "r", encoding=encoding) as file:
            answer_text = file.read()
        count +=1
        print(count)
        # Answer the questions for the current file
        current_answers = []
        for question in Questions_item01:
            answer = answer_question_longfile(question, answer_text, max_num_sentences, overlap_num_sentences)
            current_answers.append(answer)

        # Append the answers for the current file to the list
        all_answers.append(current_answers)

# Create a pandas dataframe using the list of answers and the file names
df = pd.DataFrame(all_answers, columns=Questions_item01)
#df.insert(0, "File Name", os.listdir('/content/item5'))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [22]:
df

,What is the initial fee?,What is the initial fee in dollars?,What is the initial franchise fee in dollars?,What is the standard nonrefundable initial franchise fee in dollars?
0,"$35,000","$35,000","$35,000","$35,000"
1,"$50,000","$50,000","$50,000","$50,000"
2,"$5,000","$5,000","$5,000",five thousand
3,"$25,000","$25,000","$25,000","$25,000"
4,"$45,000","$45,000","$45,000",$
5,"$20,000","$20,000","$20,000","$20,000"
6,thirty-five thousand dollars,thirty-five thousand,thirty-five thousand,one hundred and ten thousand
7,you must pay us an initial franchise fee (“ if...,"$60,000",$0,[CLS]
8,"$30,000","$30,000","$30,000","$30,000"
9,"$54,000","$54,000","$54,000","$54,000"


In [23]:
text_file = []

for filename in os.listdir('/content/item5'):
    if filename.endswith(".txt"):
      text_file.append(filename)

# add text_file to df
df['text_file'] = text_file

In [27]:
# Read the answers file
df_answers = pd.read_csv('Item 5_Answers.csv')

# Check if the filename ends with .txt, if not add .txt
df_answers['filename'] = df_answers.apply(lambda row: row['filename'] + '.txt' if row['filename'].endswith('.txt') == False else row['filename'], axis=1)

# merge df_answers with df_reordered
df_merged = pd.merge(df_answers, df, left_on='filename', right_on='text_file').drop(columns=['text_file'])
df_merged

,filename,True Initial Fee,Alternative Answer,Alternative Answer.1,Alternative Answer.2,Alternative Answer.3,Alternative Answer.4,What is the initial fee?,What is the initial fee in dollars?,What is the initial franchise fee in dollars?,What is the standard nonrefundable initial franchise fee in dollars?
0,1 McDonald's.pdf-all_items_item_5.txt,"$45,000",NaN,NaN,NaN,NaN,NaN,"$45,000","$45,000","$45,000","$45,000"
1,3 KFC.pdf-all_items_item_5.txt,"$45,000",NaN,NaN,NaN,NaN,NaN,"$45,000","$45,000","$45,000",$
2,4 Burger King.pdf-all_items_item_5.txt,"$50,000","$25,000 (conditional FSS)","$15,000 (shorter period)",NaN,NaN,NaN,"$25,000","$25,000","$50,000","$50,000"
3,5 Ace Hardware.pdf-all_items_item_5.txt,"$5,000",NaN,NaN,NaN,NaN,NaN,"$5,000","$5,000","$5,000",five thousand
4,6 Chick-fil-A.pdf-all_items_item_5.txt,$0,NaN,NaN,NaN,NaN,NaN,we do not refund any initial fee,[CLS],[CLS],[CLS]
5,7 Subway.pdf-all_items_item_5.txt,"$15,000","$7,500(conditions)",$5000 (satellite franchise fee),$1000 (short term satellite fee),NaN,NaN,"5,","$15,000","$15,000","$15,000"
6,8 Domino's.pdf-all_items_item_5.txt,"$10,000",$1500 (exisitng store transfer),NaN,NaN,NaN,NaN,initial franchise application processing fee,"$10,000",$0,$0
7,9 Circle K.pdf-all_items_item_5.txt,"$25,000","$15,000 per store(2-5 stores)","$10,000 per store(6-9store)","$7,500(10-19 stores)","$5,000 (20 stores)",NaN,"$25,000","$15,000","$15,000","$15,000"
8,10 Taco Bell.pdf-all_items_item_5.txt,"$22,500",NaN,NaN,NaN,NaN,NaN,"$22,500","$22,500","$22,500","$22,500"
9,11 Wendy's.pdf-all_items_item_5.txt,"$50,000",NaN,NaN,NaN,NaN,NaN,"$20,000",[CLS],[CLS],[CLS]


In [28]:
# Include only numbers for calculating accuracy
import numpy as np

pattern = r"\$[\d,]+"
df_merged['True Initial Fee Modified'] = df_merged['True Initial Fee'].str.extract(f'({pattern})').replace(np.nan, '$0')
for idx, question in enumerate(Questions_item01):
  df_merged['Prediction '+str(idx+1)] = df_merged[question].str.extract(f'({pattern})').replace(np.nan, '$0')
df_merged.head(15)

,filename,True Initial Fee,Alternative Answer,Alternative Answer.1,Alternative Answer.2,Alternative Answer.3,Alternative Answer.4,What is the initial fee?,What is the initial fee in dollars?,What is the initial franchise fee in dollars?,What is the standard nonrefundable initial franchise fee in dollars?,True Initial Fee Modified,Prediction 1,Prediction 2,Prediction 3,Prediction 4
0,1 McDonald's.pdf-all_items_item_5.txt,"$45,000",NaN,NaN,NaN,NaN,NaN,"$45,000","$45,000","$45,000","$45,000","$45,000","$45,000","$45,000","$45,000","$45,000"
1,3 KFC.pdf-all_items_item_5.txt,"$45,000",NaN,NaN,NaN,NaN,NaN,"$45,000","$45,000","$45,000",$,"$45,000","$45,000","$45,000","$45,000",$0
2,4 Burger King.pdf-all_items_item_5.txt,"$50,000","$25,000 (conditional FSS)","$15,000 (shorter period)",NaN,NaN,NaN,"$25,000","$25,000","$50,000","$50,000","$50,000","$25,000","$25,000","$50,000","$50,000"
3,5 Ace Hardware.pdf-all_items_item_5.txt,"$5,000",NaN,NaN,NaN,NaN,NaN,"$5,000","$5,000","$5,000",five thousand,"$5,000","$5,000","$5,000","$5,000",$0
4,6 Chick-fil-A.pdf-all_items_item_5.txt,$0,NaN,NaN,NaN,NaN,NaN,we do not refund any initial fee,[CLS],[CLS],[CLS],$0,$0,$0,$0,$0
5,7 Subway.pdf-all_items_item_5.txt,"$15,000","$7,500(conditions)",$5000 (satellite franchise fee),$1000 (short term satellite fee),NaN,NaN,"5,","$15,000","$15,000","$15,000","$15,000",$0,"$15,000","$15,000","$15,000"
6,8 Domino's.pdf-all_items_item_5.txt,"$10,000",$1500 (exisitng store transfer),NaN,NaN,NaN,NaN,initial franchise application processing fee,"$10,000",$0,$0,"$10,000",$0,"$10,000",$0,$0
7,9 Circle K.pdf-all_items_item_5.txt,"$25,000","$15,000 per store(2-5 stores)","$10,000 per store(6-9store)","$7,500(10-19 stores)","$5,000 (20 stores)",NaN,"$25,000","$15,000","$15,000","$15,000","$25,000","$25,000","$15,000","$15,000","$15,000"
8,10 Taco Bell.pdf-all_items_item_5.txt,"$22,500",NaN,NaN,NaN,NaN,NaN,"$22,500","$22,500","$22,500","$22,500","$22,500","$22,500","$22,500","$22,500","$22,500"
9,11 Wendy's.pdf-all_items_item_5.txt,"$50,000",NaN,NaN,NaN,NaN,NaN,"$20,000",[CLS],[CLS],[CLS],"$50,000","$20,000",$0,$0,$0


In [29]:
# Calculate Accuracy

accuracy_df = pd.DataFrame()
for idx, question in enumerate(Questions_item01):
  print('Question: {}\n'.format(question))
  EM, F1 = compute_EM_F1(df_merged['Prediction '+str(idx+1)], df_merged['True Initial Fee Modified'])
  accuracy_df = pd.concat([accuracy_df, pd.DataFrame([{'Question': question, 'EM Score': statistics.mean(EM), 'F1 Score': statistics.mean(F1)}])], ignore_index=True)

Question: What is the initial fee?

EM score: 0.66

F1 score: 0.66

Question: What is the initial fee in dollars?

EM score: 0.56

F1 score: 0.56

Question: What is the initial franchise fee in dollars?

EM score: 0.62

F1 score: 0.62

Question: What is the standard nonrefundable initial franchise fee in dollars?

EM score: 0.44

F1 score: 0.44



In [30]:
# Accuracy in a table format
accuracy_df

,Question,EM Score,F1 Score
0,What is the initial fee?,0.66,0.66
1,What is the initial fee in dollars?,0.56,0.56
2,What is the initial franchise fee in dollars?,0.62,0.62
3,What is the standard nonrefundable initial fra...,0.44,0.44


In [31]:
#Save the dataframe to a csv file
df.to_csv('/content/FourQuestions_v2.csv', index=False)
df_merged.to_csv('/content/FourQuestions_v2_m.csv', index=False)